In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import kendalltau, pearsonr, spearmanr

def spearsonr_pval(x,y):
    return spearmanr(x,y)[1]

data_a = pd.read_csv("LVB-75DPI-metadata-feature-table.csv")

data_not_restore = pd.read_csv("LVB-mass-not-restored.csv")
data_not_restore_list = data_not_restore.filename.to_list()
data_not_restore_list.insert(0, "MouseID")

# this is metadata info, which is extra baylor information.
# careful for the range, [a,b] mean a <= X < b. Also python is 0 based index.
data_M = data_a.iloc[:, np.r_[0, 1:8]]
data_M = data_M.set_index('MouseID')
# this is metabolites, which is features.
data_F = data_a[data_not_restore_list]
data_F = data_F.set_index('MouseID')

# get the two part of column names: metadata colum name and features column name.
M_column_list = data_M.head(0)
print(M_column_list)
F_column_list = data_F.head(0)
print(F_column_list)

df_col_list = []
df_corr_list = []
df_pval_list = []

for m_col in M_column_list:
    for f_col in F_column_list:
        col_1 = data_M[m_col]
        col_2 = data_F[f_col]
        corr_col_1_2 = col_1.corr(col_2, method='spearman')
        corr_pval_col_1_2 = col_1.corr(col_2, method=spearsonr_pval)
        df_col_list.append(m_col + "-" + f_col)
        df_corr_list.append(corr_col_1_2)
        df_pval_list.append(corr_pval_col_1_2)

print(df_col_list)
print(df_corr_list)
print(df_pval_list)

import statsmodels.api

df_final = pd.DataFrame(list(zip(df_col_list, df_corr_list, df_pval_list)), columns=['col_name','corr', 'corr_pval'])

df_final['corr'].replace('', float("NaN"), inplace=True)
df_final.dropna(subset=['corr'], inplace=True)

dr_fdr_pval = statsmodels.stats.multitest.fdrcorrection(list(df_final["corr_pval"]), method='indep')
df_final['FDR_rejected'] = dr_fdr_pval[0]
df_final['FDR_pval'] = dr_fdr_pval[1]

df_final.to_csv('LVB-75DPI-not-restored-corr-pval.csv', index=False)

Empty DataFrame
Columns: [Heart.weight..Body.weight.ratio, Liver.weight..Body.weight.ratio, Ejection.Fraction, PR.Interval..s., QTc..s., P.Amplitude..V., CD3.CD8.IFNg.]
Index: []
Empty DataFrame
Columns: [X128.018_0.344, X136.062_0.301, X136.097_0.551, X137.046_0.35, X137.046_0.42, X203.007_0.309, X256.264_3.225, X285.133_2.768, X296.258_2.976, X301.115_0.328, X301.115_0.413, X324.327_3.455, X351.217_2.741, X380.993_2.164, X397.22_2.071, X400.97_2.127, X416.337_2.79, X420.899_2.103, X426.484_2.164, X440.337_2.765, X442.353_2.823, X458.23_2.165, X466.25_2.226, X475.989_2.164, X476.24_2.164, X570.357_2.934, X580.362_2.984, X682.363_3.329, X682.364_3.313]
Index: []

[0 rows x 29 columns]
['Heart.weight..Body.weight.ratio-X128.018_0.344', 'Heart.weight..Body.weight.ratio-X136.062_0.301', 'Heart.weight..Body.weight.ratio-X136.097_0.551', 'Heart.weight..Body.weight.ratio-X137.046_0.35', 'Heart.weight..Body.weight.ratio-X137.046_0.42', 'Heart.weight..Body.weight.ratio-X203.007_0.309', 'Heart.